In [1]:
import os
os.environ['HF_DATASETS_CACHE'] = '/work/pi_dhruveshpate_umass_edu/amalgonde_umass_edu'
os.environ['CONDA_PKGS_DIRS'] = '/work/pi_dhruveshpate_umass_edu/amalgonde_umass_edu'
os.environ['CONDA_ENVS_PATH'] = '/work/pi_dhruveshpate_umass_edu/amalgonde_umass_edu'
os.environ['TRANSFORMERS_CACHE'] = '/work/pi_dhruveshpate_umass_edu/amalgonde_umass_edu'
os.environ['HF_HOME'] = '/work/pi_dhruveshpate_umass_edu/amalgonde_umass_edu'

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             trust_remote_code=True,
                                             torch_dtype=torch.float16,
                                             device_map="auto",
                                             # load_in_8bit=True
                                            )
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
    model.cuda()

/home/amalgonde_umass_edu/.conda/envs/hw1/lib/python3.9/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- modeling_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

NVIDIA GeForce RTX 2080 Ti


In [ ]:
# TEST

input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")
# input_ids = tokenizer(input_text, return_tensors="pt", max_length=1024, padding='max_length').to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=1024)
print(type(tokenizer.decode(outputs[0])), tokenizer.decode(outputs[0])) #.replace('<pad>', ''))

In [3]:
import os
import csv
import pandas as pd
import time
from tqdm import tqdm

data_path = "/work/pi_dhruveshpate_umass_edu/project_19/aishwarya/696DS-named-entity-extraction-and-linking-for-KG-construction/webnlg/data/release_v3.0_en/sample_dev_data/"
file = "webnlg_dev_sample.csv"

data = pd.read_csv(os.path.join(data_path, file))
data.columns

Index(['original_index', 'category', 'eid', 'size', 'original_triple_sets',
       'modified_triple_sets', 'comment', 'lid', 'text', 'lang', 'shape',
       'shape_type', 'dbpedia_links', 'links', 'test_category'],
      dtype='object')

In [4]:
import sys
sys.path.append("../")
import get_prompts_exp2 as get_prompts

import json

with open('/work/pi_dhruveshpate_umass_edu/project_19/aishwarya//696DS-named-entity-extraction-and-linking-for-KG-construction/webnlg/webNLG_dev_relations.json', 'r') as fp:
    relationList = json.load(fp)

In [5]:
max_tokens = 256

# lower the value, deterministic result
temperature = 0.1

# a higher value increases the chance of finding a better output
top_p = 0.9

output_file_name = "phi2_fewdhot_webnlg_dev_sample.csv"

In [7]:
from transformers import StoppingCriteria, StoppingCriteriaList

class KeywordsStoppingCriteria(StoppingCriteria):
    def __init__(self, keywords_ids:list):
        self.keywords = keywords_ids

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        if input_ids[0][-1] in self.keywords:
            return True
        return False


stop_words = ['Test', 'Test Sentence', 'Test Output']
stop_ids = [tokenizer.encode(w)[0] for w in stop_words]
stop_criteria = KeywordsStoppingCriteria(stop_ids)
stop_ids

[14402, 14402, 14402]

In [12]:
all_inputs = list(data['text'])
output_data = []
debug = False

for i in tqdm(range(data.shape[0])):
    
    # if i <= 1940:
    #     continue
    
    # prompt = zero_shot_prompt(all_inputs[i])
    # prompt = get_prompts.one_shot_prompt_v2(all_inputs[i], relationList['all_relations'])
    prompt = get_prompts.few_shot_prompt_v1(all_inputs[i])

    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
    
    start = time.time()
    outputs = model.generate(**input_ids,
                             max_new_tokens=max_tokens,
                             top_p=top_p,
                             do_sample=True,
                             temperature=temperature,
                             pad_token_id=tokenizer.eos_token_id,
                             stopping_criteria=StoppingCriteriaList([stop_criteria]),
                             )
    time_diff = time.time() - start
    # inferenceTime.append()
    
    # output2 = tokenizer.decode(outputs[0], skip_special_tokens=True)
    output2 = tokenizer.decode(outputs[0][len(input_ids[0]):], skip_special_tokens=True)
    
    if debug:
        print(output2)
        print(f"\nGT : {data['modified_triple_sets'][i]}")
        
    output_data.append([data["original_index"][i], output2, time_diff])
    
    if i % 10 == 0 and i > 1:
        with open(output_file_name, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerows(output_data)
    # break
with open(output_file_name, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(output_data)

100%|██████████| 878/878 [1:00:14<00:00,  4.12s/it]


In [ ]:
'''
Tesla V100-PCIE-16GB - phi2_dev
'''